In [30]:
import numpy as np
import pandas as pd
import os 
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
train_app=pd.read_csv("D:/Desktop/机器学习的数据集/Home Credit Default Risk/train_app_without_ohe.csv")
test_app=pd.read_csv("D:/Desktop/机器学习的数据集/Home Credit Default Risk/test_app_without_ohe.csv")
bureau=pd.read_csv("D:/Desktop/机器学习的数据集/Home Credit Default Risk/discard_missing/bureau.csv")
bureau_balance=pd.read_csv("D:/Desktop/机器学习的数据集/Home Credit Default Risk/discard_missing/bureau_balance.csv")
credit_card_balance=pd.read_csv("D:/Desktop/机器学习的数据集/Home Credit Default Risk/discard_missing/credit_card_balance.csv")
installments_payments=pd.read_csv("D:/Desktop/机器学习的数据集/Home Credit Default Risk/discard_missing/installments_payments.csv")
POS_CASH_balance=pd.read_csv("D:/Desktop/机器学习的数据集/Home Credit Default Risk/discard_missing/POS_CASH_balance.csv")
previous_application=pd.read_csv("D:/Desktop/机器学习的数据集/Home Credit Default Risk/discard_missing/previous_application.csv")
def get_mode(x):
    modes = x.mode()
    return modes.iloc[0] if not modes.empty else None

In [2]:
def ohe_convert(df, drop_first=True):
    cat_names=list(filter(lambda x:df[x].dtype in ["object","O"],df.columns))
    if cat_names!=[]:
        return  pd.get_dummies(df, columns = cat_names, drop_first=drop_first)  # pd.get_dummies() 会自动：处理字符串类别 ,生成新列名，格式为：原列名_类别值,,,保留原始数值列不动
    else:
        return  df

In [3]:
def get_apps_processed(apps):
    # 有3组外部对该用户的评分数据，现在求平均值与方差
    apps["APPS_EXT_SOURCE_MEAN"]=apps[["EXT_SOURCE_1",'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    apps["APPS_EXT_SOURCE_MEAN"]=apps["APPS_EXT_SOURCE_MEAN"].fillna(apps["APPS_EXT_SOURCE_MEAN"].mean())
    apps["APPS_EXT_SOURCE_STD"]=apps[[ "EXT_SOURCE_1",'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    apps['APPS_EXT_SOURCE_STD'] = apps['APPS_EXT_SOURCE_STD'].fillna(apps['APPS_EXT_SOURCE_STD'].mean())   # 缺失值用方方差的平均值进行填充
    # 确保你有这三列原始数据
# EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3

    apps['EXT_1_mul_2'] = apps['EXT_SOURCE_1'] * apps['EXT_SOURCE_2']
    apps['EXT_1_mul_3'] = apps['EXT_SOURCE_1'] * apps['EXT_SOURCE_3']
    apps['EXT_2_mul_3'] = apps['EXT_SOURCE_2'] * apps['EXT_SOURCE_3']
    apps['maxwell_feature_1'] = (apps['EXT_SOURCE_1'] * apps['EXT_SOURCE_3']) ** (1 / 2)

    # 3. 权重和 (Home Credit 经典公式)
    # 这个系数是 Kaggle 社区算出来的黄金比例
    apps['EXT_SOURCES_prod']  = apps['EXT_SOURCE_1'] * apps['EXT_SOURCE_2'] * apps['EXT_SOURCE_3']
    apps['EXT_WEIGHTED'] =  apps['EXT_SOURCE_2'] * 3 + apps['EXT_SOURCE_3'] * 4  +apps['EXT_SOURCE_1'] * 2 
    # 取收入的对数
    apps["Log_AMT_INCOME_TOTAL"]=np.log(apps["AMT_INCOME_TOTAL"]+1e-8)
    apps["Log_AMT_ANNUITY"]=np.log(apps["AMT_ANNUITY"]+1e-8)

    # 30天社交圈违约率
    apps['DEF_30_RATIO'] = apps['DEF_30_CNT_SOCIAL_CIRCLE'] / (apps['OBS_30_CNT_SOCIAL_CIRCLE'] + 1e-5)
    # 60天社交圈违约率
    apps['DEF_60_RATIO'] = apps['DEF_60_CNT_SOCIAL_CIRCLE'] / (apps['OBS_60_CNT_SOCIAL_CIRCLE'] +1e-5)
    # 处于 30-60 天逾期状态的人数（早期风险信号）
    apps['DEF_30_MINUS_60'] = apps['DEF_30_CNT_SOCIAL_CIRCLE'] - apps['DEF_60_CNT_SOCIAL_CIRCLE']
    # 观察人数的变化（有些人的社交圈子可能变动了？）
    apps['OBS_30_MINUS_60'] = apps['OBS_30_CNT_SOCIAL_CIRCLE'] - apps['OBS_60_CNT_SOCIAL_CIRCLE']

    # 一年内总共的查询次数
    apps["ALL_YEAR_Inquiries"]=apps["AMT_REQ_CREDIT_BUREAU_DAY"]+apps["AMT_REQ_CREDIT_BUREAU_WEEK"]+apps["AMT_REQ_CREDIT_BUREAU_MON"]+apps["AMT_REQ_CREDIT_BUREAU_QRT"]+apps["AMT_REQ_CREDIT_BUREAU_YEAR"]
    # 通过可视化分析，对于月度的查询次数分布不是仅为0，所以对于短期我们采用月度的
    apps["MON_Inquiries"]=apps["AMT_REQ_CREDIT_BUREAU_DAY"]+apps["AMT_REQ_CREDIT_BUREAU_WEEK"]+apps["AMT_REQ_CREDIT_BUREAU_MON"]
    apps["REC_Inquiries_Rate"]=np.where(apps["ALL_YEAR_Inquiries"]>0,0,apps["MON_Inquiries"]/apps["ALL_YEAR_Inquiries"])

    
    # 贷款年金额/贷款的总金额=贷款的年利率
    apps['APPS_ANNUITY_CREDIT_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_CREDIT']
    # 贷款年金额/总共的收入=年收益率？
    apps['APPS_ANNUITY_INCOME_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_INCOME_TOTAL']
    # 贷款金额/收入金额>1，说明贷入一笔比自己工资多的金额，可能是有急事？？
    apps['APPS_CREDIT_INCOME_RATIO'] = apps['AMT_CREDIT']/apps['AMT_INCOME_TOTAL']
    ## 日常消费水平/贷款金额<1，说明贷款金额不会全用在消费上
    apps['APPS_GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_CREDIT']
    #消费水平/收入情况1>1,说明入不敷出
    apps['APPS_GOODS_INCOME_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_INCOME_TOTAL']
    apps["APPS_GOODS_ANNUITY_RATIO"] = apps["AMT_GOODS_PRICE"]/apps["AMT_ANNUITY"]
    # 收入情况/家庭成员，另一个方面描述家庭的经济情况
    apps['APPS_CNT_FAM_INCOME_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['CNT_FAM_MEMBERS']
    # 工龄/贷款时的年龄，值越大，说明该客户的工作越是一种稳定的
    apps['APPS_EMPLOYED_BIRTH_DIFFER'] = apps['DAYS_EMPLOYED']-apps['DAYS_BIRTH']
    apps['APPS_EMPLOYED_BIRTH_RATIO'] = apps['DAYS_EMPLOYED']/apps['DAYS_BIRTH']
    # 收入/工龄 ，值越大说明了工龄越小，收入越高
    apps['APPS_INCOME_EMPLOYED_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_EMPLOYED']
    # 收入/贷款时的年龄，值越大代表客户的行业可能是处于青春饭行业
    apps['APPS_INCOME_BIRTH_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_BIRTH']

    apps['DAYS_REGISTRATION_DAYS_BIRTH_DIFFER'] = apps['DAYS_REGISTRATION'] - apps['DAYS_BIRTH']

    apps['DAYS_LAST_PHONE_CHANGE_DAYS_BIRTH_DIFFER'] = apps['DAYS_LAST_PHONE_CHANGE'] - apps['DAYS_BIRTH']
    apps['DAYS_REGISTRATION_DAYS_EMPLOYED_DIFFER']  = apps["DAYS_REGISTRATION"]-apps["DAYS_EMPLOYED"]


    # 另一个侧面描述经济水平
    apps['AMT_INCOME_TOTAL-d-CNT_CHILDREN']  = apps['AMT_INCOME_TOTAL']  / (apps['CNT_FAM_MEMBERS']+0.000001)
    apps['AMT_ANNUITY-d-CNT_CHILDREN']  = apps['AMT_ANNUITY']  / (apps['CNT_FAM_MEMBERS']+0.000001)
    apps['AMT_GOODS-d-CNT_CHILDREN']  = apps['AMT_GOODS_PRICE']  / (apps['CNT_FAM_MEMBERS']+0.000001)
    apps['AMT_CREDIT-d-CNT_CHILDREN']  = apps['AMT_CREDIT']  / (apps['CNT_FAM_MEMBERS']+0.000001)
    
    docs = [_f for _f in apps.columns if 'FLAG_DOC' in _f]
    apps['alldocs_kurt'] = apps[docs].kurtosis(axis=1)  # 峰度：分布尖锐程度（是否集中在少数文档上
    apps['alldocs_skew'] = apps[docs].skew(axis=1)      # 偏度 → 是否偏向于上传多文档或少文档
    apps['alldocs_mean'] = apps[docs].mean(axis=1)     
    apps['alldocs_sum']  = apps[docs].sum(axis=1)
    apps['alldocs_std']  = apps[docs].std(axis=1)

    bdg_avg  = apps.filter(regex='_AVG$').columns
    bdg_mode = apps.filter(regex='_MODE$').columns
    bdg_medi = apps.filter(regex='_MEDI$').columns[:len(bdg_avg)] # ignore FONDKAPREMONT_MODE...
    
    apps['building_score_avg_mean'] = apps[bdg_avg].mean(1)
    apps['building_score_avg_std']  = apps[bdg_avg].std(1)
    apps['building_score_avg_sum']  = apps[bdg_avg].sum(1)

    apps['building_score_mode_mean'] = apps[bdg_mode].mean(1)
    apps['building_score_mode_std']  = apps[bdg_mode].std(1)
    apps['building_score_mode_sum']  = apps[bdg_mode].sum(1)

    apps['building_score_medi_mean'] = apps[bdg_medi].mean(1)
    apps['building_score_medi_std']  = apps[bdg_medi].std(1)
    apps['building_score_medi_sum']  = apps[bdg_medi].sum(1)



    return apps
train_app=get_apps_processed(train_app)
train_app.head()
train_app=ohe_convert(train_app,drop_first=False)
bool_cols = train_app.select_dtypes(include=['bool']).columns
train_app[bool_cols] = train_app[bool_cols].astype(int)

In [4]:
test_app=get_apps_processed(test_app)
test_app=ohe_convert(test_app,drop_first=False)
bool_cols = test_app.select_dtypes(include=['bool']).columns
test_app[bool_cols] = test_app[bool_cols].astype(int)

In [5]:
installments_payments.columns

Index(['Unnamed: 0', 'SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION',
       'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT'],
      dtype='object')

In [6]:
installments_payments['NUM_INSTALMENT_RATIO'] = installments_payments['NUM_INSTALMENT_NUMBER'] / (installments_payments['AMT_PAYMENT']+1e-5)

installments_payments['DAYS_DELAY_PAYMENTS'] = installments_payments['DAYS_ENTRY_PAYMENT'] - installments_payments['DAYS_INSTALMENT']
installments_payments['AMT_DIFF'] = installments_payments['AMT_INSTALMENT'] - installments_payments['AMT_PAYMENT']

installments_payments['AMT_RATIO'] = np.where( installments_payments['AMT_INSTALMENT']==-1,0,(installments_payments['AMT_PAYMENT'] +1)/ (installments_payments['AMT_INSTALMENT'] + 1))
# 实际还的日期-应还日期
installments_payments['SK_DPD'] = installments_payments['DAYS_ENTRY_PAYMENT'] - installments_payments['DAYS_INSTALMENT']
# 大于0说明逾期了
installments_payments['INS_IS_DPD'] = installments_payments['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
installments_payments['INS_IS_DPD_UNDER_120'] = installments_payments['SK_DPD'].apply(lambda x:1 if (x > 0) & (x <120) else 0 )
installments_payments['INS_IS_DPD_OVER_120'] = installments_payments['SK_DPD'].apply(lambda x:1 if x >= 120 else 0)
installments_payments=ohe_convert(installments_payments,False)

In [7]:
install_agg_dict_1 = {
        'SK_ID_PREV':['count',"nunique"],
        'NUM_INSTALMENT_VERSION':['nunique'], 
        'NUM_INSTALMENT_NUMBER':["nunique"],
        'DAYS_ENTRY_PAYMENT':['mean', 'max', 'sum'],
        'DAYS_INSTALMENT':['mean', 'max', 'sum'],
        'AMT_INSTALMENT':['mean', 'max', 'sum'],
        'AMT_PAYMENT':['mean', 'max','sum'],

        'AMT_DIFF':['mean','min', 'max','sum'],
        "AMT_RATIO":["mean","max","min","sum"],
        'DAYS_DELAY_PAYMENTS':["max","mean","min"],
        'NUM_INSTALMENT_RATIO':["max","min","mean"],
        'SK_DPD':['mean', 'min', 'max'],
        'INS_IS_DPD':['mean', 'sum'],
        'INS_IS_DPD_UNDER_120':['mean', 'sum'],
        'INS_IS_DPD_OVER_120':['mean', 'sum']    
    }
install_pay_agg_same_application=installments_payments.groupby(["SK_ID_PREV","SK_ID_CURR"]).agg(install_agg_dict_1).reset_index()
install_pay_agg_same_application.columns=["_".join(c).strip("_") if isinstance(c,tuple) else c for c in install_pay_agg_same_application.columns]
print(install_pay_agg_same_application.columns)
# 同一个客户的所有历史贷款特征进一步汇总，得到客户全局画像。
excul_ID_cols= list(install_pay_agg_same_application.columns[~install_pay_agg_same_application.columns.isin(['SK_ID_CURR','SK_ID_PREV'])])
install_pay_agg_same_credit=install_pay_agg_same_application.groupby("SK_ID_CURR").agg(
    dict(
        zip(
            excul_ID_cols,
            ["mean"]*len(excul_ID_cols)
            )
        )
    ).reset_index()
install_pay_agg_same_credit=install_pay_agg_same_credit.rename(
    columns=dict(
        zip(
            excul_ID_cols,
            list(map(lambda x: "AVG_"+x,excul_ID_cols))
        )
    )
)
#install_pay_agg_same_credit.drop("AVG_Unnamed: 0",axis=1,inplace=True)
install_pay_agg_same_credit.head(20)

Index(['SK_ID_PREV', 'SK_ID_CURR', 'SK_ID_PREV_count', 'SK_ID_PREV_nunique',
       'NUM_INSTALMENT_VERSION_nunique', 'NUM_INSTALMENT_NUMBER_nunique',
       'DAYS_ENTRY_PAYMENT_mean', 'DAYS_ENTRY_PAYMENT_max',
       'DAYS_ENTRY_PAYMENT_sum', 'DAYS_INSTALMENT_mean', 'DAYS_INSTALMENT_max',
       'DAYS_INSTALMENT_sum', 'AMT_INSTALMENT_mean', 'AMT_INSTALMENT_max',
       'AMT_INSTALMENT_sum', 'AMT_PAYMENT_mean', 'AMT_PAYMENT_max',
       'AMT_PAYMENT_sum', 'AMT_DIFF_mean', 'AMT_DIFF_min', 'AMT_DIFF_max',
       'AMT_DIFF_sum', 'AMT_RATIO_mean', 'AMT_RATIO_max', 'AMT_RATIO_min',
       'AMT_RATIO_sum', 'DAYS_DELAY_PAYMENTS_max', 'DAYS_DELAY_PAYMENTS_mean',
       'DAYS_DELAY_PAYMENTS_min', 'NUM_INSTALMENT_RATIO_max',
       'NUM_INSTALMENT_RATIO_min', 'NUM_INSTALMENT_RATIO_mean', 'SK_DPD_mean',
       'SK_DPD_min', 'SK_DPD_max', 'INS_IS_DPD_mean', 'INS_IS_DPD_sum',
       'INS_IS_DPD_UNDER_120_mean', 'INS_IS_DPD_UNDER_120_sum',
       'INS_IS_DPD_OVER_120_mean', 'INS_IS_DPD_OVER_120_sum'

,SK_ID_CURR,AVG_SK_ID_PREV_count,AVG_SK_ID_PREV_nunique,AVG_NUM_INSTALMENT_VERSION_nunique,AVG_NUM_INSTALMENT_NUMBER_nunique,AVG_DAYS_ENTRY_PAYMENT_mean,AVG_DAYS_ENTRY_PAYMENT_max,AVG_DAYS_ENTRY_PAYMENT_sum,AVG_DAYS_INSTALMENT_mean,AVG_DAYS_INSTALMENT_max,...,AVG_NUM_INSTALMENT_RATIO_mean,AVG_SK_DPD_mean,AVG_SK_DPD_min,AVG_SK_DPD_max,AVG_INS_IS_DPD_mean,AVG_INS_IS_DPD_sum,AVG_INS_IS_DPD_UNDER_120_mean,AVG_INS_IS_DPD_UNDER_120_sum,AVG_INS_IS_DPD_OVER_120_mean,AVG_INS_IS_DPD_OVER_120_sum
0,100001,3.500000,1.0,1.500000,3.500000,-2280.916667,-2242.000000,-7682.500000,-2275.000000,-2237.500000,...,0.000595,-5.916667,-18.000000,2.500000,0.166667,0.500000,0.166667,0.500000,0.000000,0.000000
1,100002,19.000000,1.0,2.000000,19.000000,-315.421053,-49.000000,-5993.000000,-295.000000,-25.000000,...,0.000992,-20.421053,-31.000000,-12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,100003,8.333333,1.0,1.333333,8.333333,-1171.781746,-1063.333333,-11544.333333,-1164.333333,-1054.333333,...,0.000351,-7.448413,-12.000000,-4.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,100004,3.000000,1.0,2.000000,3.000000,-761.666667,-727.000000,-2285.000000,-754.000000,-724.000000,...,0.000281,-7.666667,-11.000000,-3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,100005,9.000000,1.0,2.000000,9.000000,-609.555556,-470.000000,-5486.000000,-586.000000,-466.000000,...,0.000888,-23.555556,-37.000000,1.000000,0.111111,1.000000,0.111111,1.000000,0.000000,0.000000
5,100006,5.333333,1.0,1.333333,5.333333,-285.966667,-208.000000,-1448.666667,-260.666667,-195.666667,...,0.000354,-25.300000,-37.666667,-12.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,100007,13.200000,1.0,1.200000,12.600000,-1090.768539,-917.400000,-13625.600000,-1087.881319,-915.200000,...,0.998238,-2.887220,-16.800000,3.800000,0.287912,3.200000,0.287912,3.200000,0.000000,0.000000
7,100008,8.750000,1.0,1.250000,8.500000,-1301.861742,-892.250000,-10830.750000,-1318.931818,-1209.500000,...,0.001325,17.070076,-22.000000,322.750000,0.022727,0.250000,0.000000,0.000000,0.022727,0.250000
8,100009,6.375000,1.0,1.000000,6.375000,-894.558333,-815.375000,-5510.625000,-885.125000,-804.500000,...,0.000435,-9.433333,-15.500000,-5.125000,0.031250,0.125000,0.031250,0.125000,0.000000,0.000000
9,100010,10.000000,1.0,1.000000,10.000000,-915.900000,-774.000000,-9159.000000,-904.000000,-769.000000,...,0.000200,-11.900000,-30.000000,-5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [8]:
def attrbution_app_and_credit(df_1):
    ##############
    df_1.drop('Unnamed: 0',axis=1,inplace=True)
    df=df_1.copy()
    df_1.set_index("SK_ID_CURR",inplace=True)
    binary_names=list(filter(lambda x: df[x].dtype in ["object","o"],df.columns))
    print(binary_names)
    binary_series=df_1[binary_names]
    binary_agg = binary_series.groupby("SK_ID_CURR").agg(get_mode).reset_index()
    binary_agg.columns = ["SK_ID_CURR"] + [f"MODE_{col}" for col in binary_agg.columns[1:]]
    id_cols=list(df.columns[:2])   # 也即是SK_ID_CURR和SK_ID_PREV
    print(id_cols)
    contnames = [col for col in df.columns if col not in id_cols and col not in binary_names]
    print(contnames)
    # 采取和上述一样聚合的方式
    ##########

    agg_dict={}
    for col in contnames:
        agg_dict[col]=["mean","sum","max"]
    for col in binary_names:
        agg_dict[col]=["count","nunique"]

    df_agg_same_app = df.groupby(id_cols).agg(
       agg_dict
    ).reset_index()
    # 展平列名（处理多级索引）
    df_agg_same_app.columns = [
        '_'.join(col).strip('_') if isinstance(col, tuple) else col 
        for col in df_agg_same_app.columns
    ]
    ########################
    curr_col=df.columns[1]  # 也就是SK_ID_CURR

    feature_cols = [c for c in df_agg_same_app.columns if c not in id_cols]
    second_agg_dict={}
    for col in feature_cols:
        second_agg_dict[col]="mean"
    df_agg_same_credit = df_agg_same_app.groupby([curr_col]).agg(
        second_agg_dict
    ).reset_index()

    rename_dict = {}
    for c in feature_cols:
        rename_dict[c]=f"AVG_{c}"
    df_agg_same_credit=df_agg_same_credit.rename(
        columns=rename_dict
    )

    df_final=pd.merge(df_agg_same_credit,binary_agg,on=curr_col,how="left")

    return df_final



In [9]:
POS_CASH_balance.columns

Index(['Unnamed: 0', 'SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE',
       'CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE', 'NAME_CONTRACT_STATUS',
       'SK_DPD', 'SK_DPD_DEF'],
      dtype='object')

In [10]:
POS_CASH_balance['CNT_INSTALMENT-m-CNT_INSTALMENT_FUTURE'] = POS_CASH_balance['CNT_INSTALMENT'] - POS_CASH_balance['CNT_INSTALMENT_FUTURE']
POS_CASH_balance['CNT_INSTALMENT_FUTURE-d-CNT_INSTALMENT'] = POS_CASH_balance['CNT_INSTALMENT_FUTURE'] / (1+POS_CASH_balance['CNT_INSTALMENT'])
POS_CASH_balance['POS_IS_DPD'] = POS_CASH_balance['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
POS_CASH_balance['POS_IS_DPD_UNDER_120'] = POS_CASH_balance['SK_DPD'].apply(lambda x:1 if (x > 0) & (x <120) else 0 )
POS_CASH_balance['POS_IS_DPD_OVER_120'] = POS_CASH_balance['SK_DPD'].apply(lambda x:1 if x >= 120 else 0)
POS_CASH_balance=ohe_convert(POS_CASH_balance,False)

In [11]:
pos_bal_agg_same_credit=attrbution_app_and_credit(POS_CASH_balance.copy())
pos_bal_agg_same_credit.head()

[]
['SK_ID_PREV', 'SK_ID_CURR']
['MONTHS_BALANCE', 'CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE', 'SK_DPD', 'SK_DPD_DEF', 'CNT_INSTALMENT-m-CNT_INSTALMENT_FUTURE', 'CNT_INSTALMENT_FUTURE-d-CNT_INSTALMENT', 'POS_IS_DPD', 'POS_IS_DPD_UNDER_120', 'POS_IS_DPD_OVER_120', 'NAME_CONTRACT_STATUS_Active', 'NAME_CONTRACT_STATUS_Completed and  Others (new) ']


,SK_ID_CURR,AVG_MONTHS_BALANCE_mean,AVG_MONTHS_BALANCE_sum,AVG_MONTHS_BALANCE_max,AVG_CNT_INSTALMENT_mean,AVG_CNT_INSTALMENT_sum,AVG_CNT_INSTALMENT_max,AVG_CNT_INSTALMENT_FUTURE_mean,AVG_CNT_INSTALMENT_FUTURE_sum,AVG_CNT_INSTALMENT_FUTURE_max,...,AVG_POS_IS_DPD_UNDER_120_max,AVG_POS_IS_DPD_OVER_120_mean,AVG_POS_IS_DPD_OVER_120_sum,AVG_POS_IS_DPD_OVER_120_max,AVG_NAME_CONTRACT_STATUS_Active_mean,AVG_NAME_CONTRACT_STATUS_Active_sum,AVG_NAME_CONTRACT_STATUS_Active_max,AVG_NAME_CONTRACT_STATUS_Completed and Others (new) _mean,AVG_NAME_CONTRACT_STATUS_Completed and Others (new) _sum,AVG_NAME_CONTRACT_STATUS_Completed and Others (new) _max
0,100001,-74.750000,-326.500000,-73.0,4.000000,18.000000,4.0,1.375000,6.5,3.0,...,0.5,0.0,0.0,0.0,0.775000,3.500000,1.0,0.225000,1.000000,1.000000
1,100002,-10.000000,-190.000000,-1.0,24.000000,456.000000,24.0,15.000000,285.0,24.0,...,0.0,0.0,0.0,0.0,1.000000,19.000000,1.0,0.000000,0.000000,0.000000
2,100003,-39.166667,-408.666667,-35.0,9.791667,94.333333,10.0,5.666667,54.0,10.0,...,0.0,0.0,0.0,0.0,0.916667,8.666667,1.0,0.083333,0.666667,0.666667
3,100004,-25.500000,-102.000000,-24.0,3.750000,15.000000,4.0,2.250000,9.0,4.0,...,0.0,0.0,0.0,0.0,0.750000,3.000000,1.0,0.250000,1.000000,1.000000
4,100005,-20.000000,-220.000000,-15.0,11.727273,129.000000,12.0,7.181818,79.0,12.0,...,0.0,0.0,0.0,0.0,0.818182,9.000000,1.0,0.181818,2.000000,1.000000


In [31]:
credit_card_balance.columns

Index(['Unnamed: 0', 'SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE',
       'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
       'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
       'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE',
       'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT',
       'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT',
       'CNT_INSTALMENT_MATURE_CUM', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF'],
      dtype='object')

In [ ]:
##当月的余额/实际可用额度，值越大。说明余额越多，花的少
credit_card_balance['BALANCE_LIMIT_RATIO'] = np.where(credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL'] == 0,-1,  # 或 0，或 -1，视业务而定
    credit_card_balance['AMT_BALANCE'] / credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL']
)
 ##当月消费金额/当月实际可用额度，值越大，花的越多
credit_card_balance['DRAWING_LIMIT_RATIO'] = np.where(credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL'] == 0,0,
    credit_card_balance['AMT_DRAWINGS_CURRENT'] / credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL']
)

credit_card_balance['AMT_BALANCE-d-AMT_DRAWINGS_CURRENT'] = credit_card_balance['AMT_BALANCE'] / (credit_card_balance['AMT_DRAWINGS_CURRENT']+1e-5)
credit_card_balance['AMT_RECEIVABLE_RECEIVABLE_PRINCIPAL'] = credit_card_balance['AMT_TOTAL_RECEIVABLE'] - credit_card_balance['AMT_RECEIVABLE_PRINCIPAL']
credit_card_balance['AMT_RECEIVABLE_PRINCIPAL_RECEIVABLE'] = credit_card_balance['AMT_RECEIVABLE_PRINCIPAL'] / (credit_card_balance['AMT_TOTAL_RECEIVABLE']+1e-5)

credit_card_balance['CARD_IS_DPD'] = credit_card_balance['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
credit_card_balance['CARD_IS_DPD_UNDER_120'] = credit_card_balance['SK_DPD'].apply(lambda x:1 if (x > 0) & (x <120) else 0 )
credit_card_balance['CARD_IS_DPD_OVER_120'] = credit_card_balance['SK_DPD'].apply(lambda x:1 if x >= 120 else 0)
credit_card_balance=ohe_convert(credit_card_balance,False)

In [13]:
ccard_bal_agg_same_credit=attrbution_app_and_credit(credit_card_balance.copy())
ccard_bal_agg_same_credit.head()

[]
['SK_ID_PREV', 'SK_ID_CURR']
['MONTHS_BALANCE', 'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY', 'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM', 'SK_DPD', 'SK_DPD_DEF', 'BALANCE_LIMIT_RATIO', 'DRAWING_LIMIT_RATIO', 'CARD_IS_DPD', 'CARD_IS_DPD_UNDER_120', 'CARD_IS_DPD_OVER_120', 'NAME_CONTRACT_STATUS_Active', 'NAME_CONTRACT_STATUS_Others (new)']


,SK_ID_CURR,AVG_MONTHS_BALANCE_mean,AVG_MONTHS_BALANCE_sum,AVG_MONTHS_BALANCE_max,AVG_AMT_BALANCE_mean,AVG_AMT_BALANCE_sum,AVG_AMT_BALANCE_max,AVG_AMT_CREDIT_LIMIT_ACTUAL_mean,AVG_AMT_CREDIT_LIMIT_ACTUAL_sum,AVG_AMT_CREDIT_LIMIT_ACTUAL_max,...,AVG_CARD_IS_DPD_UNDER_120_max,AVG_CARD_IS_DPD_OVER_120_mean,AVG_CARD_IS_DPD_OVER_120_sum,AVG_CARD_IS_DPD_OVER_120_max,AVG_NAME_CONTRACT_STATUS_Active_mean,AVG_NAME_CONTRACT_STATUS_Active_sum,AVG_NAME_CONTRACT_STATUS_Active_max,AVG_NAME_CONTRACT_STATUS_Others (new)_mean,AVG_NAME_CONTRACT_STATUS_Others (new)_sum,AVG_NAME_CONTRACT_STATUS_Others (new)_max
0,100006,-3.5,-21.0,-1.0,0.000000,0.000,0.00,270000.000000,1620000.0,270000.0,...,0.0,0.0,0.0,0.0,1.000000,6.0,1.0,0.000000,0.0,0.0
1,100011,-38.5,-2849.0,-2.0,54482.111149,4031676.225,189000.00,164189.189189,12150000.0,180000.0,...,0.0,0.0,0.0,0.0,1.000000,74.0,1.0,0.000000,0.0,0.0
2,100013,-48.5,-4656.0,-1.0,18159.919219,1743352.245,161420.22,131718.750000,12645000.0,157500.0,...,1.0,0.0,0.0,0.0,1.000000,96.0,1.0,0.000000,0.0,0.0
3,100021,-10.0,-170.0,-2.0,0.000000,0.000,0.00,675000.000000,11475000.0,675000.0,...,0.0,0.0,0.0,0.0,0.411765,7.0,1.0,0.588235,10.0,1.0
4,100023,-7.5,-60.0,-4.0,0.000000,0.000,0.00,135000.000000,1080000.0,225000.0,...,0.0,0.0,0.0,0.0,1.000000,8.0,1.0,0.000000,0.0,0.0


In [14]:
previous_application.columns

Index(['Unnamed: 0', 'SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE',
       'AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_GOODS_PRICE',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE',
       'CODE_REJECT_REASON', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY',
       'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE',
       'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY', 'CNT_PAYMENT',
       'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION'],
      dtype='object')

In [15]:
# 申请的贷款金额-最终发放的贷款金额
previous_application['PREV_CREDIT_DIFF'] = previous_application['AMT_APPLICATION'] - previous_application['AMT_CREDIT']
# 申请的贷款金额-在过往申请过程中的商品价格，这个代表什么意思？？
previous_application['PREV_GOODS_DIFF'] = previous_application['AMT_APPLICATION'] - previous_application['AMT_GOODS_PRICE']
previous_application["PREV_ANNUITY_DIFF"]=previous_application["AMT_APPLICATION"]-previous_application["AMT_ANNUITY"]
previous_application["PREV_CREDIT_GOODS_DIFF"]=previous_application["AMT_CREDIT"]-previous_application["AMT_GOODS_PRICE"]
previous_application["PREV_CREDIT_ANNUITY_DIFF"]=previous_application["AMT_CREDIT"]-previous_application["AMT_ANNUITY"]
previous_application["PREV_GOODS_ANNUITY_DIFF"]=previous_application["AMT_GOODS_PRICE"]-previous_application["AMT_ANNUITY"]

# 申请的贷款金额/最终发放的贷款金额，大于1，说明申请的金额要比发放的金额要高
previous_application['PREV_CREDIT_APPL_RATIO'] =np.where(previous_application['AMT_APPLICATION']==0 ,0,previous_application['AMT_CREDIT']/previous_application['AMT_APPLICATION'])
previous_application['PREV_ANNUITY_APPL_RATIO'] = previous_application['AMT_ANNUITY']/(previous_application['AMT_APPLICATION']+1e-5)
# 商品价格/贷款金额，值大于1，说明消费水平不低
previous_application['PREV_GOODS_APPL_RATIO'] = np.where(previous_application['AMT_APPLICATION']==0,0,previous_application['AMT_GOODS_PRICE']/previous_application['AMT_APPLICATION'])


previous_application["PREV_APPL_CREDIT_RATIO"]=previous_application["AMT_APPLICATION"]/(previous_application["AMT_CREDIT"]+1e-5)
previous_application["PRV_GOODS_CREDIT_RATIO"]=previous_application["AMT_GOODS_PRICE"]/(previous_application["AMT_CREDIT"]+1e-5)
previous_application["PREV_ANNUITY_CREDIT_RATIO"]=previous_application["AMT_ANNUITY"]/(previous_application["AMT_CREDIT"]+1e-5)

previous_application["PREV_APPL_GOODS_RATIO"]=previous_application["AMT_APPLICATION"]/(previous_application["AMT_GOODS_PRICE"]+1e-5)
previous_application["PREV_CREDIT_GOODS_RATIO"]=previous_application["AMT_CREDIT"]/(previous_application["AMT_GOODS_PRICE"]+1e-5)
previous_application["PREV_ANNUITY_GOODS_RATIO"]=previous_application["AMT_ANNUITY"]/(previous_application["AMT_GOODS_PRICE"]+1e-5)

previous_application["PREV_APPL_ANNUIYY_RATIO"]=previous_application["AMT_APPLICATION"]/(previous_application["AMT_ANNUITY"]+1e-5)
previous_application["PREV_CREDIT_ANNUITY_RATIO"]=previous_application["AMT_CREDIT"]/(previous_application["AMT_ANNUITY"]+1e-5)
previous_application["PREV_GOODS_ANNUITY_RATIO"]=previous_application["AMT_GOODS_PRICE"]/(previous_application["AMT_ANNUITY"]+1e-5)

# 每年的贷款* 利息=贷款期间的总利息
all_pay = previous_application['AMT_ANNUITY'] * previous_application['CNT_PAYMENT']
credit = previous_application['AMT_CREDIT']
payment = previous_application['CNT_PAYMENT']

# 安全计算：避免 inf
condition = (credit != 0) & (payment != 0)
numerator = all_pay / credit - 1
denominator = payment

rate = np.where(condition, numerator / denominator, 0)

# 限制范围
rate = np.clip(rate, 0, 1.0)  # 保证利率在 [0, 1]

previous_application['PREV_INTERESTS_RATE'] = rate
previous_application=ohe_convert(previous_application,False)

In [16]:
prev_app_agg_same_credit=attrbution_app_and_credit(previous_application.copy())
prev_app_agg_same_credit.head()

[]
['SK_ID_PREV', 'SK_ID_CURR']
['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_GOODS_PRICE', 'HOUR_APPR_PROCESS_START', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'PREV_CREDIT_DIFF', 'PREV_GOODS_DIFF', 'PREV_ANNUITY_DIFF', 'PREV_CREDIT_GOODS_DIFF', 'PREV_CREDIT_ANNUITY_DIFF', 'PREV_GOODS_ANNUITY_DIFF', 'PREV_CREDIT_APPL_RATIO', 'PREV_ANNUITY_APPL_RATIO', 'PREV_GOODS_APPL_RATIO', 'PREV_APPL_CREDIT_RATIO', 'PRV_GOODS_CREDIT_RATIO', 'PREV_ANNUITY_CREDIT_RATIO', 'PREV_APPL_GOODS_RATIO', 'PREV_CREDIT_GOODS_RATIO', 'PREV_ANNUITY_GOODS_RATIO', 'PREV_APPL_ANNUIYY_RATIO', 'PREV_CREDIT_ANNUITY_RATIO', 'PREV_GOODS_ANNUITY_RATIO', 'PREV_INTERESTS_RATE', 'NAME_CONTRACT_TYPE_Cash loans', 'NAME_CONTRACT_TYPE_Consumer loans', 'NAME_CONTRACT_TYPE_Revolving loans and  Others (new) ', 'WEEKDAY_APPR_PROCESS_START_FRIDAY', 'WEEKDAY_APPR_PROCESS_START_MONDAY', 'WEEKDAY_APPR_PROCESS_START_SATURDAY', 'WEEKDAY_APPR_PROCESS_START_SUNDAY', 'WEEKDAY_APPR_PROCESS_START_THURSDAY', 'WEEKDAY_APPR_PROC

,SK_ID_CURR,AVG_AMT_ANNUITY_mean,AVG_AMT_ANNUITY_sum,AVG_AMT_ANNUITY_max,AVG_AMT_APPLICATION_mean,AVG_AMT_APPLICATION_sum,AVG_AMT_APPLICATION_max,AVG_AMT_CREDIT_mean,AVG_AMT_CREDIT_sum,AVG_AMT_CREDIT_max,...,AVG_PRODUCT_COMBINATION_Others (new)_max,AVG_PRODUCT_COMBINATION_POS household with interest_mean,AVG_PRODUCT_COMBINATION_POS household with interest_sum,AVG_PRODUCT_COMBINATION_POS household with interest_max,AVG_PRODUCT_COMBINATION_POS industry with interest_mean,AVG_PRODUCT_COMBINATION_POS industry with interest_sum,AVG_PRODUCT_COMBINATION_POS industry with interest_max,AVG_PRODUCT_COMBINATION_POS mobile with interest_mean,AVG_PRODUCT_COMBINATION_POS mobile with interest_sum,AVG_PRODUCT_COMBINATION_POS mobile with interest_max
0,100001,3951.000,3951.000,3951.000,24835.50,24835.50,24835.50,23787.00,23787.00,23787.00,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0
1,100002,9251.775,9251.775,9251.775,179055.00,179055.00,179055.00,179055.00,179055.00,179055.00,...,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,100003,56553.990,56553.990,56553.990,435436.50,435436.50,435436.50,484191.00,484191.00,484191.00,...,0.0,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.0,0.0,0.0
3,100004,5357.250,5357.250,5357.250,24282.00,24282.00,24282.00,20106.00,20106.00,20106.00,...,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,100005,8031.600,8031.600,8031.600,22308.75,22308.75,22308.75,20076.75,20076.75,20076.75,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.5,0.5


In [32]:
bureau.columns

Index(['Unnamed: 0', 'SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE',
       'CREDIT_CURRENCY', 'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE',
       'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'CNT_CREDIT_PROLONG',
       'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE'],
      dtype='object')

In [ ]:
bureau['DAYS_CREDIT_ENDDATE_CREDIT_DIFF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_CREDIT']
bureau['DAYS_CREDIT_UPDATE_CREDIT_DIFF'] = bureau['DAYS_CREDIT_UPDATE'] - bureau['DAYS_CREDIT']
bureau['DAYS_CREDIT_UPDATE_CREDIT_ENDDATE_DIFF'] = bureau['DAYS_CREDIT_UPDATE'] - bureau['DAYS_CREDIT_ENDDATE']
bureau['DAYS_CREDIT_UPDATE_ENDDATE_FACT_DIFF'] = bureau['DAYS_CREDIT_UPDATE'] - bureau['DAYS_ENDDATE_FACT']

bureau['AMT_CREDIT_SUM_CREDIT_SUM_DEBT_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
bureau['AMT_CREDIT_SUM_DEBT_CREDIT_SUM_RATIO'] = bureau['AMT_CREDIT_SUM_DEBT'] / bureau['AMT_CREDIT_SUM']
bureau['AMT_CREDIT_SUM_CREDIT_DIFF_RATIO'] = bureau['AMT_CREDIT_SUM_CREDIT_SUM_DEBT_DIFF'] /(bureau['AMT_CREDIT_SUM_LIMIT']+1e-5)
bureau['AMT_CREDIT_SUM_DEBT_RATIO'] = bureau['AMT_CREDIT_SUM_DEBT'] / (bureau['AMT_CREDIT_SUM_LIMIT']+1e-5)
bureau['AMT_CREDIT_SUM_DEBT_SUM'] =bureau['AMT_CREDIT_SUM_DEBT'] + bureau['AMT_CREDIT_SUM_LIMIT']
bureau['AMT_CREDIT_SUM_RATIO'] = bureau['AMT_CREDIT_SUM'] / (bureau['AMT_CREDIT_SUM_DEBT_SUM']+1e-5)

# 在Home Credit机构申请的贷款剩余有效天数-终止天数
bureau['BUREAU_ENDDATE_FACT_DIFF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']
# 申请贷款所需天数-终止天数
bureau['BUREAU_CREDIT_FACT_DIFF'] = bureau['DAYS_CREDIT'] - bureau['DAYS_ENDDATE_FACT']
#
bureau['BUREAU_CREDIT_ENDDATE_DIFF'] = bureau['DAYS_CREDIT'] - bureau['DAYS_CREDIT_ENDDATE']
# 总债务/总的贷款金额
bureau['BUREAU_CREDIT_DEBT_RATIO']=np.where(bureau['AMT_CREDIT_SUM']==0,0,bureau['AMT_CREDIT_SUM_DEBT']/bureau['AMT_CREDIT_SUM'])
#bureau['BUREAU_CREDIT_DEBT_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
# 总债务-总贷款金额
bureau['BUREAU_CREDIT_DEBT_DIFF'] = bureau['AMT_CREDIT_SUM_DEBT'] - bureau['AMT_CREDIT_SUM']
# 逾期的用户记为1
bureau['BUREAU_IS_DPD'] = bureau['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x > 0 else 0)
# 逾期天数大于120天的用户记为1
bureau['BUREAU_IS_DPD_OVER120'] = bureau['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x >120 else 0)
#bureau=ohe_convert(bureau,False)
bool_cols_3 = bureau.select_dtypes(include=['bool']).columns
bureau[bool_cols_3] = bureau[bool_cols_3].astype(int)
bureau_balance=ohe_convert(bureau_balance,False)

In [18]:
## bureau_balance 只有一列是SK_ID_BUREAU，还有两行是MONTHS_BALANCE，STATUS
print(bureau_balance.columns)
#bureau_balance.drop("Unnamed: 0",axis=1,inplace=True)
binary_names_1 =  bureau_balance.select_dtypes(include=['object',"O"]).columns.tolist()

contnames_1 = list(set(bureau_balance.columns) - set(binary_names_1))
contnames_1.remove('SK_ID_BUREAU')
print(contnames_1)
bureau_bal_agg_dict={}
for c in contnames_1:
    bureau_bal_agg_dict[c]=["sum","max","mean"]
if not binary_names_1:
    pass
else:
    for c in binary_names_1:
        bureau_bal_agg_dict[c]=["count","nunique",]

bureau_bal_agg_same_credit = bureau_balance.groupby('SK_ID_BUREAU').agg(
    bureau_bal_agg_dict
).reset_index()

bureau_bal_agg_same_credit.columns = [
        '_'.join(col).strip('_') if isinstance(col, tuple) else col 
        for col in bureau_bal_agg_same_credit.columns
    ]
bureau_bal_agg_same_credit.head()

Index(['Unnamed: 0', 'SK_ID_BUREAU', 'MONTHS_BALANCE', 'STATUS_0', 'STATUS_C',
       'STATUS_X and  Others (new) '],
      dtype='object')
['STATUS_0', 'STATUS_X and  Others (new) ', 'Unnamed: 0', 'MONTHS_BALANCE', 'STATUS_C']


,SK_ID_BUREAU,STATUS_0_sum,STATUS_0_max,STATUS_0_mean,STATUS_X and Others (new) _sum,STATUS_X and Others (new) _max,STATUS_X and Others (new) _mean,Unnamed: 0_sum,Unnamed: 0_max,Unnamed: 0_mean,MONTHS_BALANCE_sum,MONTHS_BALANCE_max,MONTHS_BALANCE_mean,STATUS_C_sum,STATUS_C_max,STATUS_C_mean
0,5001709,0,False,0.000000,11,True,0.113402,2529740697,26079849,26079801.0,-4656,0,-48.0,86,True,0.886598
1,5001710,5,True,0.060241,30,True,0.361446,236247631,2846398,2846357.0,-3403,0,-41.0,48,True,0.578313
2,5001711,3,True,0.750000,1,True,0.250000,11385602,2846402,2846400.5,-6,0,-1.5,0,False,0.000000
3,5001712,10,True,0.526316,0,False,0.000000,54081828,2846421,2846412.0,-171,0,-9.0,9,True,0.473684
4,5001713,0,False,0.000000,22,True,1.000000,31135247,1415249,1415238.5,-231,0,-10.5,0,False,0.000000


In [19]:
bureau_combined = bureau.merge(bureau_bal_agg_same_credit, on='SK_ID_BUREAU', how='left')
bureau_combined['BUREAU_BAL_NA'] = 1  # 创建一个新的列
bureau_combined.loc[bureau_combined.isnull().any(axis=1), 'BUREAU_BAL_NA'] = 0   # 检测在合并之后的，这个客户的某笔历史贷款是否有详细的历史行为记录。。如果0代表没有历史记录，1代表有历史记录
bureau_combined.fillna(0, inplace=True)
bureau_combined.head()
bureau_combined.drop("Unnamed: 0",axis=1,inplace=True)
bureau_combined

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,...,Unnamed: 0_sum,Unnamed: 0_max,Unnamed: 0_mean,MONTHS_BALANCE_sum,MONTHS_BALANCE_max,MONTHS_BALANCE_mean,STATUS_C_sum,STATUS_C_max,STATUS_C_mean,BUREAU_BAL_NA
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,0,91323.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
1,215354,5714463,Active and Others (new),currency 1,-208,0,1075.0,-897.0,0,225000.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
2,215354,5714464,Active and Others (new),currency 1,-203,0,528.0,-897.0,0,464323.50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
3,215354,5714465,Active and Others (new),currency 1,-203,0,-330.0,-897.0,0,90000.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
4,215354,5714466,Active and Others (new),currency 1,-629,0,1197.0,-897.0,0,2700000.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active and Others (new),currency 1,-44,0,-30.0,-897.0,0,11250.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,0,38130.84,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,0,15570.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,0,36000.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0


In [20]:
binary_names_3 = list(filter(lambda x: bureau_combined[x].dtype in ["object","o"] or bureau_combined[x].nunique==2, 
                        bureau_combined.columns))
print(binary_names_3)
contnames_3 = list(set(bureau_combined.columns) - set(binary_names_3))
contnames_3.remove('SK_ID_CURR')
contnames_3.remove('SK_ID_BUREAU')
print("type of contnames_3:", type(contnames_3))

bureau_agg_dict={}
bureau_agg_dict['SK_ID_BUREAU'] = 'count'
for c in contnames_3:
    bureau_agg_dict[c]=["mean","max","sum"]
for c in binary_names_3:
    bureau_agg_dict[c]=["count","nunique"]
bureau_combined_agg = bureau_combined.groupby('SK_ID_CURR').agg(
    bureau_agg_dict
).reset_index()
bureau_combined_agg.columns=["_".join(c).strip("_")  if isinstance(c,tuple) else c  
                             for c in bureau_combined_agg.columns]
bureau_combined_agg=bureau_combined_agg.rename(columns={"SK_ID_BUREAU_count":"SK_ID_BUREAU"})
bureau_combined_agg.head()

['CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'CREDIT_TYPE', 'STATUS_0_max', 'STATUS_X and  Others (new) _max', 'STATUS_C_max']
type of contnames_3: <class 'list'>


,SK_ID_CURR,SK_ID_BUREAU,BUREAU_CREDIT_ENDDATE_DIFF_mean,BUREAU_CREDIT_ENDDATE_DIFF_max,BUREAU_CREDIT_ENDDATE_DIFF_sum,CNT_CREDIT_PROLONG_mean,CNT_CREDIT_PROLONG_max,CNT_CREDIT_PROLONG_sum,STATUS_X and Others (new) _mean_mean,STATUS_X and Others (new) _mean_max,...,CREDIT_CURRENCY_count,CREDIT_CURRENCY_nunique,CREDIT_TYPE_count,CREDIT_TYPE_nunique,STATUS_0_max_count,STATUS_0_max_nunique,STATUS_X and Others (new) _max_count,STATUS_X and Others (new) _max_nunique,STATUS_C_max_count,STATUS_C_max_nunique
0,100001,7,-817.428571,-243.0,-5722.0,0.0,0,0,0.222109,0.500000,...,7,1,7,1,7,1,7,2,7,2
1,100002,8,-529.750000,227.0,-4238.0,0.0,0,0,0.417614,0.562500,...,8,1,8,2,8,1,8,2,8,2
2,100003,4,-856.250000,-152.0,-3425.0,0.0,0,0,0.000000,0.000000,...,4,1,4,2,4,1,4,1,4,1
3,100004,2,-378.500000,-26.0,-757.0,0.0,0,0,0.000000,0.000000,...,2,1,2,1,2,1,2,1,2,1
4,100005,3,-630.000000,-184.0,-1890.0,0.0,0,0,0.136752,0.333333,...,3,1,3,2,3,1,3,2,3,2


In [21]:
train_final = train_app.merge(prev_app_agg_same_credit, on='SK_ID_CURR', how='left', suffixes=(None, '_prevapp')).merge(
    ccard_bal_agg_same_credit, on='SK_ID_CURR', how='left', suffixes=(None, '_ccard')).merge(
    pos_bal_agg_same_credit, on='SK_ID_CURR', how='left', suffixes=(None, '_posbal')) .merge(
    install_pay_agg_same_credit, on='SK_ID_CURR', how='left', suffixes=(None, '_installpay')).merge(
    bureau_combined_agg, on='SK_ID_CURR', how='left', suffixes=(None, '_bureau'))
train_final.drop("Unnamed: 0",axis=1,inplace=True)
train_final.head()

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,CREDIT_CURRENCY_count,CREDIT_CURRENCY_nunique,CREDIT_TYPE_count,CREDIT_TYPE_nunique,STATUS_0_max_count,STATUS_0_max_nunique,STATUS_X and Others (new) _max_count,STATUS_X and Others (new) _max_nunique,STATUS_C_max_count,STATUS_C_max_nunique
0,100002,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,-3648.0,-2120,...,8.0,1.0,8.0,2.0,8.0,1.0,8.0,2.0,8.0,2.0
1,100003,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,-1186.0,-291,...,4.0,1.0,4.0,2.0,4.0,1.0,4.0,1.0,4.0,1.0
2,100004,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,-4260.0,-2531,...,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0
3,100006,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,-9833.0,-2437,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,-4311.0,-3458,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [22]:
test_final = test_app.merge(prev_app_agg_same_credit, on='SK_ID_CURR', how='left', suffixes=(None, '_prevapp')).merge(
    ccard_bal_agg_same_credit, on='SK_ID_CURR', how='left', suffixes=(None, '_ccard')).merge(
    pos_bal_agg_same_credit, on='SK_ID_CURR', how='left', suffixes=(None, '_posbal')) .merge(
    install_pay_agg_same_credit, on='SK_ID_CURR', how='left', suffixes=(None, '_installpay')).merge(
    bureau_combined_agg, on='SK_ID_CURR', how='left', suffixes=(None, '_bureau'))
test_final.drop("Unnamed: 0",axis=1,inplace=True)
test_final.head()


,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,CREDIT_CURRENCY_count,CREDIT_CURRENCY_nunique,CREDIT_TYPE_count,CREDIT_TYPE_nunique,STATUS_0_max_count,STATUS_0_max_nunique,STATUS_X and Others (new) _max_count,STATUS_X and Others (new) _max_nunique,STATUS_C_max_count,STATUS_C_max_nunique
0,100001,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,-5170.0,-812,...,7.0,1.0,7.0,1.0,7.0,1.0,7.0,2.0,7.0,2.0
1,100005,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,-9118.0,-1623,...,3.0,1.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,2.0
2,100013,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,-2175.0,-3503,...,4.0,1.0,4.0,2.0,4.0,2.0,4.0,1.0,4.0,2.0
3,100028,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,-2000.0,-4208,...,12.0,1.0,12.0,2.0,12.0,2.0,12.0,2.0,12.0,2.0
4,100038,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,-4000.0,-4262,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
train_final['Bureau_flag'] = 1
test_final['Bureau_flag'] = 1
train_final.loc[train_final.SK_ID_BUREAU.isnull(),'Bureau_flag']=0
test_final.loc[test_final.SK_ID_BUREAU.isnull(),'Bureau_flag']=0
# 同样的在 在HC上没有过往记录的记为0
train_final.fillna(0,inplace=True)
test_final.fillna(0,inplace=True)

In [24]:
train_final.to_csv("D:/Desktop/机器学习的数据集/Home Credit Default Risk/train_fianl_with_ohe.csv")
test_final.to_csv("D:/Desktop/机器学习的数据集/Home Credit Default Risk/test_final_with_ohe.csv")

In [25]:
del bureau,bureau_balance,install_pay_agg_same_application,install_pay_agg_same_credit,installments_payments,prev_app_agg_same_credit,previous_application,credit_card_balance,ccard_bal_agg_same_credit


In [26]:
del bureau_bal_agg_same_credit,bureau_combined,bureau_combined_agg

In [27]:
del test_app,test_final,train_app,train_final,condition,credit,numerator,payment,POS_CASH_balance,

In [28]:
del pos_bal_agg_same_credit,denominator,all_pay